In [ ]:
# Connect your Google Drive so that you download config files and store models
# Change to your home directory on Google Drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

In [ ]:
# Download sample training, test, eval & config files needed for training model
# A sample model is also downloaded for those wishing to skip the training step
!wget https://github.com/adaptNMT/mtdev/raw/main/rnn.zip

In [ ]:
# Unpack the files and create the directory structure
!unzip -o rnn.zip

In [ ]:
# Display GPU details provided by Google
gpu_info = !nvidia-smi
print(gpu_info)

# Check if version of python >= 3.8
!python --version

# Check if Pytorch > 2.0 is installed
!apt list | grep torch

In [ ]:
# Install the chosen MT engine required for translation (OpenNMT used here)
!pip install OpenNMT-py

In [ ]:
# Build the vocabulary using the hyperparameters set in rnn.yaml
%cd rnn/
!onmt_build_vocab -config data/rnn.yaml -n_sample=-1

**TRAIN MODEL**

In [ ]:
# The model is trained using the hyperparameters stored in rnn.yaml
!onmt_train -config data/rnn.yaml

**TRANSLATION**

In [ ]:
# OPTIONAL CELL:
# Download a sample model if you haven't completed the previous training step
!pip install -q gdown
import gdown
id = "10i7HAA9UvWPDmi4iTT04E1rb1y4BmXIX&export=download&confirm=t"
output="models/model_step_10000.pt"
gdown.download(id=id, output=output, quiet=False)

In [ ]:
# Using the OpenNMT command, translate the English Test file into Irish
!onmt_translate  -model models/model_step_10000.pt \
		  -src data/src-test.txt \
		  -output data/pred.txt -verbose

**EVALUATION**

In [ ]:
# Install the library which will be used for carrying out the evaluation
# The translations generated in pred.text are compared with the reference
# translations provided in tgt-test.txt
!pip3 install sacrebleu[ja]
%cd data
!echo "++ using sacrebleu ++" | tee -a experiment_log.txt
!sacrebleu tgt-test.txt < pred_rnn.txt -m bleu --force | tee -a experiment_log.txt